# 1. Training the Yolo model

In [ ]:
import ultralytics
ultralytics.checks()

In [2]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=Violence.yaml epochs=50 imgsz=416 plots=True

# 2. Using the YOLO model to filter violent clips

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('D:\\Yolo\\runs\\detect\\train\\weights\\best.pt')

# Input video file path
input_video_path = 'D:\\Yolo\\test\\videos'
output_clip_path = 'D:\\Yolo\\test\\result'

# Create the output folder (if it doesn't exist)
os.makedirs(output_clip_path, exist_ok=True)

# Get all video files
video_files = [f for f in os.listdir(input_video_path) if f.endswith('.mp4')]

for video_file in video_files:
    cap = cv2.VideoCapture(os.path.join(input_video_path, video_file))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_count = 0
    clip_number = 1
    clip_frames = []
    start_time = None
    last_detection_time = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection on the current frame
        results = model(frame)

        # Extract detection results
        detections = results[0].boxes.data.cpu().numpy()
        current_time = frame_count / fps
        if len(detections) > 0:
            # If an object is detected, save the current frame
            clip_frames.append(frame)
            if start_time is None:
                start_time = current_time
            last_detection_time = current_time
        else:
            # If no object is detected but the time interval is less than 2 seconds, save the current frame
            if last_detection_time is not None and current_time - last_detection_time < 2:
                clip_frames.append(frame)
            else:
                # If the time interval is greater than 2 seconds and there are accumulated frames, save these frames as a video clip
                if len(clip_frames) > 0:
                    end_time = current_time if last_detection_time is None else last_detection_time
                    start_time_formatted = f'{int(start_time // 60):02d}_{int(start_time % 60):02d}'
                    end_time_formatted = f'{int(end_time // 60):02d}_{int(end_time % 60):02d}'
                    clip_output_path = os.path.join(output_clip_path, f'{os.path.splitext(video_file)[0]}_clip_{clip_number}_{start_time_formatted}_to_{end_time_formatted}.mp4')
                    out = cv2.VideoWriter(clip_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
                    for clip_frame in clip_frames:
                        out.write(clip_frame)
                    out.release()
                    clip_frames = []
                    clip_number += 1
                    start_time = None
                    last_detection_time = None

        frame_count += 1

    # Process the remaining frames at the end of the video
    if len(clip_frames) > 0:
        end_time = current_time if last_detection_time is None else last_detection_time
        start_time_formatted = f'{int(start_time // 60):02d}_{int(start_time % 60):02d}'
        end_time_formatted = f'{int(end_time // 60):02d}_{int(end_time % 60):02d}'
        clip_output_path = os.path.join(output_clip_path, f'{os.path.splitext(video_file)[0]}_clip_{clip_number}_{start_time_formatted}_to_{end_time_formatted}.mp4')
        out = cv2.VideoWriter(clip_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
        for clip_frame in clip_frames:
            out.write(clip_frame)
        out.release()

    cap.release()

cv2.destroyAllWindows()

